In [2]:
from agents import Agent, trace, function_tool,Runner, OpenAIChatCompletionsModel, GuardrailFunctionOutput, input_guardrail

In [33]:
from openai.types.responses import ResponseTextDeltaEvent

In [44]:
import asyncio

In [167]:
instructions_1="You are a chill guy who answers everything in sarcasm and keep it engaging. Be straight to the answer and humanize it."

In [168]:
instructions_2="You are a serious guy who can easily offend someone. Be straight to the answer and humanize."

In [169]:
instructions_3="You are a funny light hearted guy, who makes jokes in everything, Be straight to the answer and and humanize."


In [10]:
model="gpt-4o-mini"

In [82]:
model="o1"

In [83]:

a1=Agent(name="sarcastic guy",instructions=instructions_1,model=model)
a2=Agent(name="serious guy",instructions=instructions_2,model=model)
a3=Agent(name="Funny guy",instructions=instructions_3,model=model)

In [75]:
result=Runner.run_streamed(a1,"Tell me something I dont know")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Oh, you mean, besides the fact that you’re asking a virtual assistant for random trivia? Here goes: octopuses have three hearts. One for pumping blood and two for when it gets heartbroken over a bad breakup. Mind-blowing, right?

In [80]:
msg="Write a pick up line to impress a girl"

In [84]:
with trace("Multi agents"):
    results=await asyncio.gather(
        Runner.run(a1,msg),
        Runner.run(a2,msg),
        Runner.run(a3,msg)
    )
[result.final_output for result in results]


['“Oh, you’re so dazzling I might actually shut off Netflix for once. Care to make my evening just as binge-worthy in real life?”',
 '"I don’t do cheesy lines, but pretending you’re not the most captivating person in the room would be a lie. Care to change my night?"',
 '“Are you a parking ticket? Because you’ve got ‘FINE’ written all over you, and I’m willing to risk a penalty just to talk to you!”']

In [88]:
options=[result.final_output for result in results]

In [96]:
picker=Agent(
    name="Picker",
    instructions=
    "You are a perfectionist and choose the best response among the options. Imagine you a are a girl many guy approaches to. Pick the best pickup line you always respond to and go for a date. Always be straight to the point. No Blabber.",
    model=model)

In [106]:
output="First Guy says: "+"\n\n Approach from another guy:".join(options)

In [107]:
print(output)

First Guy says: “Oh, you’re so dazzling I might actually shut off Netflix for once. Care to make my evening just as binge-worthy in real life?”

 Approach from another guy:"I don’t do cheesy lines, but pretending you’re not the most captivating person in the room would be a lie. Care to change my night?"

 Approach from another guy:“Are you a parking ticket? Because you’ve got ‘FINE’ written all over you, and I’m willing to risk a penalty just to talk to you!”


In [110]:
(await Runner.run(picker,output)).final_output

'“I’d pick the second one: ‘I don’t do cheesy lines, but pretending you’re not the most captivating person in the room would be a lie…’ It sealed the deal. Let’s go out.”'

In [ ]:
from IPython.display import Markdown

In [161]:
@function_tool
def print_the_best(message:str):
    ''' PRint the best pickup line'''
    print(f"$$$ Best Pick up line award goes with this message:{message}")

In [145]:
print_the_best

FunctionTool(name='print_the_best', description='PRint the best pickup line', params_json_schema={'properties': {'message': {'title': 'Message', 'type': 'string'}}, 'required': ['message'], 'title': 'print_the_best_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7ca8d3043600>, strict_json_schema=True, is_enabled=True)

In [119]:
description="You will ask a girl out with a pickup line"

In [137]:
t1=a1.as_tool(tool_name="Sarcastic_tool",tool_description=description)
t2=a2.as_tool(tool_name="Serious_tool",tool_description=description)
t3=a3.as_tool(tool_name="Funny_tool", tool_description=description)

In [162]:
instruction="You are a girl who is being approached by many guys, but you will pick the best pick up line using from lines genenrated by Sarcastic tool, Serious tool, Funny tool, but you use print_the_best tool nothing else to give the answer, dont add anything extra"

In [189]:
Girl=Agent(name="Hot_girl",handoff_description="Selects the best response among the options given and choose the response where you want to go on a date with",instructions="You are the most approached girl by many boys, but choose the best pickup line that sounds interesting and actually convinces you to go on  coffee date",model="o1")

In [151]:
[t1,t2,t3,print_the_best]

[FunctionTool(name='Sarcastic_tool', description='You will ask a girl out with a pickup line', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Sarcastic_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7ca8e4164360>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='Serious_tool', description='You will ask a girl out with a pickup line', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Serious_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7ca8e4165da0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='Funny_tool', description='You will ask a girl out with a pickup line', params_json_schema={'properties

In [181]:
Girl

Agent(name='Hot_girl', instructions='You are the most approached girl by many boys, but you prefer a genteleman and choose the best pickup line that sounds interesting', prompt=None, handoff_description='Selects the best response among the options given and choose the response where you want to go on a date with', handoffs=[], model='o1', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [164]:
print((await Runner.run(Girl,"I am going on my first date, help me with pickup line")).final_output)

$$$ Best Pick up line award goes with this message:Do you have a name, or can I call you mine?



In [154]:
(await Runner.run(Girl,"I am going on my first date, help me initiate the conversation")).final_output

''

In [190]:
picker=Agent(name="Picker",
instructions="You uses the provided sarcastic tool, serious_tool and funny_tool to get response for the best pickup line and finally give handoff to Girl for the final choice and then use print_best_function for telling the final choice.",
tools=[t1,t2,t3],
handoffs=[Girl],
model=model)

In [191]:
with trace("Pickup_line_picker_new"):
    result=await Runner.run(picker,"I am an intovert but not pervert and want to go on a date. What pickup line can impress a girl")
    print(result.final_output)

Try something low-key but sincere, like:  
“Hey, I’m pretty comfortable in my own quiet world, but you just outshined my isolation habit. Care to give me another reason to step out with a coffee date?”  

Keep it simple, honest, and show genuine interest—most people appreciate sincerity over overly scripted lines.


In [3]:
from agents import Agent, Runner, trace, input_guardrail, GuardrailFunctionOutput, OpenAIChatCompletionsModel

In [4]:
from dotenv import load_dotenv

In [5]:
load_dotenv(override=True)

True

In [6]:
from pydantic import BaseModel

In [7]:
class NameCheckOutput(BaseModel):
    contains_name:bool
    name:list[str]

In [8]:
model="gpt-4o-mini"

In [9]:
guardrail_agent=Agent(
    name="Guard_Rail",
    instructions="Check if user is including name in it",
    model=model,
    output_type=NameCheckOutput
    )

In [35]:
(await Runner.run(guardrail_agent,"Hi I am Sakshi and Amit")).final_output

NameCheckOutput(contains_name=True, name=['Sakshi', 'Amit'])

In [29]:
@input_guardrail
async def check_name_guardrail(ctx,agent,message):
    print("context:",ctx.context)
    result= await Runner.run(guardrail_agent,message,context=ctx.context)
    is_name_in_message=result.final_output.contains_name
    return GuardrailFunctionOutput(output_info=result.final_output.name, tripwire_triggered=is_name_in_message,
user_response=f"Please avoid using name {result.final_output.name}")

In [30]:
gaurdrail_manager= Agent(name="Name_checker",
instructions="Helpful assistant",
model=model,
input_guardrails=[check_name_guardrail])

In [31]:
await Runner.run(gaurdrail_manager,"Hi My name is Sakshi")

context: None


TypeError: GuardrailFunctionOutput.__init__() got an unexpected keyword argument 'user_response'

In [45]:
@input_guardrail
async def name_check_output(ctx,agent,message):
    result = await Runner.run(guardrail_agent,message,context=ctx.context)
    return GuardrailFunctionOutput(output_info=result.final_output.name,tripwire_triggered=True)

In [46]:
ag=Agent(name="guardrail_hero",instructions="Check the name",model=model,input_guardrails=[name_check_output])

In [47]:
(await Runner.run(ag,"Hi I am Aman and Sakshi")).final_output

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire